<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goals" data-toc-modified-id="Goals-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goals</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Converting-to-qiime2-artifacts" data-toc-modified-id="Converting-to-qiime2-artifacts-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Converting to qiime2 artifacts</a></span><ul class="toc-item"><li><span><a href="#OTU-table" data-toc-modified-id="OTU-table-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>OTU table</a></span></li><li><span><a href="#Metadata" data-toc-modified-id="Metadata-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Taxonomy" data-toc-modified-id="Taxonomy-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Taxonomy</a></span></li><li><span><a href="#Phylogeny" data-toc-modified-id="Phylogeny-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Phylogeny</a></span></li></ul></li><li><span><a href="#Calculating-diversity" data-toc-modified-id="Calculating-diversity-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculating diversity</a></span></li><li><span><a href="#Converting-to-R-objects" data-toc-modified-id="Converting-to-R-objects-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Converting to R objects</a></span><ul class="toc-item"><li><span><a href="#alpha-diversity" data-toc-modified-id="alpha-diversity-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>alpha-diversity</a></span><ul class="toc-item"><li><span><a href="#Compiling-into-a-list" data-toc-modified-id="Compiling-into-a-list-6.1.1"><span class="toc-item-num">6.1.1&nbsp;&nbsp;</span>Compiling into a list</a></span></li></ul></li><li><span><a href="#beta-diversity" data-toc-modified-id="beta-diversity-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>beta-diversity</a></span><ul class="toc-item"><li><span><a href="#Compiling-into-a-list" data-toc-modified-id="Compiling-into-a-list-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Compiling into a list</a></span></li></ul></li></ul></li><li><span><a href="#Final-files" data-toc-modified-id="Final-files-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Final files</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goals

* Calculating alpha & beta diversity via qiime2
* First, aggregating OTUs at the genus level

# Var

In [1]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2/'

# 16S-arch phyloseq object (one sample per individual)
physeq_arch_IndD_file = '/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/phyloseq/physeq_arch_IndD.RDS'

# misc
tmp_dir = file.path(work_dir, 'qiime2_files')
conda_env = 'qiime2-2019.10'

# Init

In [2]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(phyloseq)
library(ape)
library(LeyLabRMisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
df.dims()
make_dir(work_dir)
make_dir(tmp_dir)

Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2/ 
Directory already exists: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files 


# Converting to qiime2 artifacts

In [21]:
# loading phyloseq object & filtering to just methanogens
physeq = readRDS(physeq_arch_IndD_file) %>%
    subset_taxa(grepl('Methano|Thermoplas', Class)) %>%
    prune_samples(sample_sums(.) > 0, .) 
physeq

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 1542 taxa and 184 samples ]
sample_data() Sample Data:       [ 184 samples by 88 sample variables ]
tax_table()   Taxonomy Table:    [ 1542 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 1542 tips and 1539 internal nodes ]

## OTU table

In [22]:
# converting OTU table
tmp = physeq %>% phyloseq2df(otu_table)
tmp = cbind(tmp %>% dplyr::select(OTU),
            tmp %>% dplyr::select(-OTU))
tmp

OTU,F14_Common_Bream,F35_Red_Deer,F36_Red_Deer,F44_Fallow_Deer,F45_Red_Deer,F46_Red_Deer,F47_Red_Deer,F48_Red_Deer,F53_Mouflon,⋯,X133_Domestic_Goat,X134_Domestic_Goat,X281_Tammar_Wallaby,X3_Alpine_Chamois,X357_Northern_Tamandua,X359_Brown_throated_Sloth,X403_Chacma_Baboon,X405_Nile_Crocodile,X431_Quagga,X67_European_Roe
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0e97060fe4b9be17bddeedcd27826b04,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,5,0,0,0,0
a86c26bd7f16d93e38b101dbd2614a07,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
9449f372c36cd2d8994d79b091425d42,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
467eeae6997d3667b5d07a76bc1bcdfd,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [23]:
# OTU table value summary
rowSums(tmp[,2:ncol(tmp)]) %>% summary %>% print
colSums(tmp[,2:ncol(tmp)]) %>% summary %>% print

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    1.0     3.0    14.0   268.9    78.0 17228.0 
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   24.0   590.5  1378.5  2253.1  2981.8 32294.0 


In [24]:
# writing out table
otu_table_file = file.path(tmp_dir, 'physeq_otu-table_meth.tsv')
write.table(tmp, otu_table_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', otu_table_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu-table_meth.tsv 


In [25]:
# converting otu to biom
otu_biom_file = file.path(tmp_dir, 'physeq_otu-table_meth.biom')
cmd = sprintf('biom convert -i %s -o %s --to-hdf5', otu_table_file, otu_biom_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "biom convert -i /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu-table_meth.tsv -o /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu-table_meth.biom --to-hdf5"


NULL

In [26]:
# converting biom to qiime2 artifact
otu_art_file = file.path(tmp_dir, 'physeq_otu_meth.qza')
cmd = sprintf('qiime tools import --type FeatureTable[Frequency] --input-path %s --output-path %s', 
              otu_biom_file, otu_art_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "qiime tools import --type FeatureTable[Frequency] --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu-table_meth.biom --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu_meth.qza"
Imported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu-table_meth.biom as BIOMV210DirFmt to /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu_meth.qza

NULL

## Metadata

In [27]:
# converting sample metadata
tmp = physeq %>% phyloseq2df(sample_data)
tmp = cbind(tmp %>% dplyr::select(SampleID),
            tmp %>% dplyr::select(-SampleID))
colnames(tmp)[1] = '#SampleID'
tmp$host_class = tmp$class
tmp 

#SampleID,host_subject_id,host_german_name,host_common_name,scientific_name,scientific_name_subspecies,host_taxid,class,order,family,⋯,i.CH4_L.d_mean,i.CH4_L.d_sd,i.CH4_L.kgBM_mean,i.CH4_L.kgBM_sd,i.CH4_g.d_mean,i.CH4_g.d_sd,i.CH4_kJ.d_mean,i.CH4_kJ.d_sd,Sample,host_class
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
F14_Common_Bream,F14,Brachse,Common.Bream,Abramis_brama,NA,38527,Actinopterygii,Cypriniformes,Cyprinidae,⋯,NA,NA,NA,NA,NA,NA,NA,NA,F14_Common_Bream,Actinopterygii
F35_Red_Deer,F35,Rothirsch,Red.Deer,Cervus_elaphus,NA,9860,Mammalia,Artiodactyla,Cervidae,⋯,52.24132,27.18306,0.4858472,0.1686955,38.67117,21.30162,2066.942,1075.505,F35_Red_Deer,Mammalia
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
X431_Quagga,431,Steppenzebra,Quagga,Equus_quagga,NA,89252,Mammalia,Perissodactyla,Equidae,⋯,NA,NA,NA,NA,NA,NA,NA,NA,X431_Quagga,Mammalia
X67_European_Roe,67,Reh,European.Roe,Capreolus_capreolus,NA,9858,Mammalia,Artiodactyla,Cervidae,⋯,NA,NA,NA,NA,NA,NA,NA,NA,X67_European_Roe,Mammalia


In [28]:
# writing out table
sample_data_file = file.path(tmp_dir, 'physeq_sample-data_meth.tsv')
write.table(tmp, sample_data_file, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', sample_data_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_sample-data_meth.tsv 


## Taxonomy

In [29]:
# converting taxonomy
tmp = physeq %>% phyloseq2df(tax_table)
tmp = cbind(tmp %>% dplyr::select(OTU),
            tmp %>% dplyr::select(-OTU))
tmp = tmp %>%
    unite(taxonomy, Domain, Phylum, Class, Order, Family, Genus, sep=';') %>%
    mutate(taxonomy = gsub(';+$','', taxonomy))
tmp

OTU,taxonomy,Species
<chr>,<chr>,<chr>
0e97060fe4b9be17bddeedcd27826b04,Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter,unclassified
a86c26bd7f16d93e38b101dbd2614a07,Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter,unclassified
⋮,⋮,⋮
9449f372c36cd2d8994d79b091425d42,Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter,unclassified
467eeae6997d3667b5d07a76bc1bcdfd,Archaea;Euryarchaeota;Methanobacteria;Methanobacteriales;Methanobacteriaceae;Methanobrevibacter,unclassified


In [30]:
# writing out table
tax_table_file = file.path(tmp_dir, 'physeq_tax-table_meth.tsv')
write.table(tmp, tax_table_file, sep='\t', quote=FALSE, row.names=FALSE, col.names=FALSE)
cat('File written:', tax_table_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_tax-table_meth.tsv 


In [31]:
# converting tsv to artifact
tax_art_file = file.path(tmp_dir, 'physeq_tax_meth.qza')
cmd = sprintf('qiime tools import --type FeatureData[Taxonomy] --input-format HeaderlessTSVTaxonomyFormat --input-path %s --output-path %s', 
              tax_table_file, tax_art_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "qiime tools import --type FeatureData[Taxonomy] --input-format HeaderlessTSVTaxonomyFormat --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_tax-table_meth.tsv --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_tax_meth.qza"
Imported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_tax-table_meth.tsv as HeaderlessTSVTaxonomyFormat to /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_tax_meth.qza

NULL

## Phylogeny

In [32]:
tree_file = file.path(tmp_dir, 'physeq_16S_meth.nwk')
write.tree(physeq %>% phy_tree, tree_file)
cat('File written:', tree_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_16S_meth.nwk 


In [33]:
# convert to artifact
tree_art_file = file.path(tmp_dir, 'physeq_16Stree_meth.qza')
cmd = sprintf('qiime tools import --type Phylogeny[Rooted] --input-path %s --output-path %s', 
              tree_file, tree_art_file)
print(cmd)
bash_job(cmd, conda_env)

[1] "qiime tools import --type Phylogeny[Rooted] --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_16S_meth.nwk --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_16Stree_meth.qza"
Imported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_16S_meth.nwk as NewickDirectoryFormat to /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_16Stree_meth.qza

NULL

# Calculating diversity

In [34]:
# core-metrics: phylogenetic diversity function

## output 
core_metrics_phylo_dir = file.path(tmp_dir, 'core_metrics_phylo_meth')
unlink(core_metrics_phylo_dir, recursive=TRUE)
log_file = file.path(tmp_dir, 'core-metrics-phylo_meth.log')


## command
cmd = "qiime diversity core-metrics-phylogenetic \\
  --i-phylogeny %s \\
  --i-table %s \\
  --m-metadata-file %s \\
  --output-dir %s \\
  --p-sampling-depth %s \\
  2> %s 1>&2"

cmd = sprintf(cmd,
              tree_art_file,
              otu_art_file, 
              sample_data_file, 
              core_metrics_phylo_dir,
              200, log_file)

## run
cat(cmd)
bash_job(cmd, conda_env)

qiime diversity core-metrics-phylogenetic \
  --i-phylogeny /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_16Stree_meth.qza \
  --i-table /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_otu_meth.qza \
  --m-metadata-file /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/physeq_sample-data_meth.tsv \
  --output-dir /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth \
  --p-sampling-depth 200 \
  2> /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core-metrics-phylo_meth.log 1>&2

NULL

# Converting to R objects

## alpha-diversity

In [35]:
# phylo vectors
files = list.files(core_metrics_phylo_dir, pattern='*_vector.qza', full.names=TRUE)
files

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/evenness_vector.qza"     
[2] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/faith_pd_vector.qza"     
[3] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/observed_otus_vector.qza"
[4] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/shannon_vector.qza"

In [36]:
# phylo vectors
cmd = "qiime tools export --output-path %s --input-path %s"

for(f in files){
    # export qza to tsv
    cmd_tmp = sprintf(cmd, core_metrics_phylo_dir, f)
    cat(cmd_tmp, '\n')
    bash_job(cmd_tmp, conda_env)    
          
    # convert tsv to data.frame RDS
    exported_file = file.path(core_metrics_phylo_dir, 'alpha-diversity.tsv')
    df = read.delim(exported_file, sep='\t', row.names=1)
    final_file = gsub('_vector.+', '.RDS', f)
    final_file = file.path(work_dir, basename(final_file))
    saveRDS(df, final_file)
    cat('File written:', final_file, '\n\n')
}

qiime tools export --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/evenness_vector.qza 
Exported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/evenness_vector.qza as AlphaDiversityDirectoryFormat to directory /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_methFile written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//evenness.RDS 

qiime tools export --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/

### Compiling into a list

In [37]:
alpha_div = list(
   'evenness' = 'evenness.RDS',
   'observed' = 'observed_otus.RDS',
   'shannon' = 'shannon.RDS',
   'faith_pd' = 'faith_pd.RDS'
)

alpha_div_rds = list()
for(n in names(alpha_div)){
    x = readRDS(file.path(work_dir, alpha_div[[n]]))
    alpha_div_rds[[n]] = x
}

alpha_div_rds_file = file.path(work_dir, 'alpha_div_meth.RDS')
saveRDS(alpha_div_rds, alpha_div_rds_file)
cat('File written:', alpha_div_rds_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//alpha_div_meth.RDS 


## beta-diversity

In [38]:
# phylo distance matrices
files = list.files(core_metrics_phylo_dir, pattern='*matrix.qza', full.names=TRUE)
files

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/bray_curtis_distance_matrix.qza"       
[2] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/jaccard_distance_matrix.qza"           
[3] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/unweighted_unifrac_distance_matrix.qza"
[4] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/weighted_unifrac_distance_matrix.qza"

In [39]:
# phylo distance matrices
cmd = "qiime tools export --output-path %s --input-path %s"

for(f in files){
    # export qza to tsv
    cmd_tmp = sprintf(cmd, core_metrics_phylo_dir, f)
    cat(cmd_tmp, '\n') 
    bash_job(cmd_tmp, conda_env)    
          
    # convert tsv to data.frame RDS
    exported_file = file.path(core_metrics_phylo_dir, 'distance-matrix.tsv')
    d = read.delim(exported_file, sep='\t', row.names=1) %>% as.dist
    final_file = gsub('_distance_matrix.+', '_meth.RDS', f)
    final_file = file.path(work_dir, basename(final_file))
    saveRDS(d, final_file)
    cat('\nFile written:', final_file, '\n\n')
}

qiime tools export --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth --input-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/bray_curtis_distance_matrix.qza 
Exported /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth/bray_curtis_distance_matrix.qza as DistanceMatrixDirectoryFormat to directory /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//qiime2_files/core_metrics_phylo_meth
File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//bray_curtis_meth.RDS 

qiime tools export --output-path /ebio/abt3_projects/Georg_animal_feces/data/16S_arc

### Compiling into a list

In [40]:
beta_div = list(
   'bray_curtis' = 'bray_curtis_meth.RDS',
   'jaccard' = 'jaccard_meth.RDS',
   'unweighted_unifrac' = 'unweighted_unifrac_meth.RDS',
   'weighted_unifrac' = 'weighted_unifrac_meth.RDS'
)

beta_div_rds = list()
for(n in names(beta_div)){
    x = readRDS(file.path(work_dir, beta_div[[n]]))
    beta_div_rds[[n]] = x
}

beta_div_rds_file = file.path(work_dir, 'beta_div_meth.RDS')
saveRDS(beta_div_rds, beta_div_rds_file)
cat('File written:', beta_div_rds_file, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//beta_div_meth.RDS 


# Final files

In [41]:
print(alpha_div_rds_file)
print(beta_div_rds_file)

[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//alpha_div_meth.RDS"
[1] "/ebio/abt3_projects/Georg_animal_feces/data/16S_arch/MiSeq-Runs-116-122-126-189-190/LLA/merged/diversity-qiime2//beta_div_meth.RDS"


# sessionInfo

In [42]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/phyloseq/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.6 ape_5.4           phyloseq_1.30.0   ggplot2_3.3.1    
[5] tidyr_1.1.0       dplyr_1.0.0      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4.6        lattice_0.20-41     Biostrings_2.54.0  
 [4] digest_0.6.25       foreach_1.5.0       IRdisplay_0.7